In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import *
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.neighbors import KNeighborsRegressor
import gc
import xgboost as xgb

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv('./data/feature/train.csv',parse_dates=['date','time_start'],dtype={'link_ID':str})
train_df = train_df.sort_values(by=['link_ID','time_start'])

In [6]:
train_df = train_df[train_df.date>=pd.to_datetime('20160302',format='%Y%m%d')]
train_df.fillna(method='ffill',inplace=True)
meta_train_data = train_df[(train_df.month==3)]
meta_val_data = train_df[(train_df.month==4)&(train_df.time_start_hour==8)]
stk_train_data = train_df[(train_df.month==4)]
stk_val_data = train_df[(train_df.month==5)&(train_df.time_start_hour==8)]
#val_data = train_df[(train_df.time_start_hour==8)&(train_df.month==5)]
#val_index = val_data.index
#test_data = train_df[(train_df.month==6)&(train_df.time_start_hour==8)]
#test_index = test_data.index
#rm_index = val_index.append(test_index)
#rm_index = train_df.index.isin(rm_index)
#train_data = train_df[~rm_index]
#train_data = train_data[train_df.time_start<pd.Timestamp('2016-05-01-00')]
#test_id = test_data.link_ID.values
#test_time = test_data.time_start.values

In [7]:
col_list = meta_train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).columns.tolist()
meta_train_y = meta_train_data.travel_time.values
meta_val_y = meta_val_data.travel_time.values
stk_train_y = stk_train_data.travel_time.values
stk_val_y = stk_val_data.travel_time.values
meta_train_data = meta_train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
meta_val_data = meta_val_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
stk_train_data = stk_train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
stk_val_data = stk_val_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values

In [12]:
dtrain_meta = lgb.Dataset(meta_train_data,meta_train_y,feature_name=col_list,categorical_feature=['link_ID','class'])
dval_meta = lgb.Dataset(meta_val_data,meta_val_y,feature_name=col_list,categorical_feature=['link_ID','class'])
dtrain_meta_xgb = xgb.DMatrix(meta_train_data,meta_train_y,feature_names=col_list)
dval_meta_xgb = xgb.DMatrix(meta_val_data,meta_val_y,feature_names=col_list)

In [9]:
def mape_eval(pred,dtrain):
    label = dtrain.get_label()
    error = (np.abs(pred-label)/label).mean()
    return 'mape',error,False

In [18]:
def mape_eval_xgb(pred,dtrain):
    label = dtrain.get_label()
    error = (np.abs(pred-label)/label).mean()
    return 'mape',error

In [10]:
def WriteTxt(dir, x, delimiter=" "):
    text_file = open(dir, "w")
    nrow = np.alen(x)
    ncol = np.alen(x[0])
    for i in range(nrow):
        temp = str(x[i, 0])
        for j in range(1, ncol):
            temp = temp + delimiter + str(x[i, j])
        if i + 1 != nrow:
            temp = temp + "\n"
        text_file.write(temp)
    text_file.close()

In [11]:
param_dart = {'num_leaves':50, 'num_trees':1200,'learning_rate':0.005,'objective':'regression_l1',
              'num_threads':8,'feature_fraction':0.65,'bagging_fraction':0.6,'gaussian_eta':1,
              'bagging_freq':1,'max_bin':20,'lambda_l1':10,'boosting':'dart','device':'cpu',
             'min_data_in_leaf':300}
dart = lgb.train(param_dart, dtrain_meta,feval = mape_eval,valid_sets=[dval_meta],verbose_eval=True,num_boost_round=1500,early_stopping_rounds=20)

[1]	valid_0's mape: 2.47384
Train until valid scores didn't improve in 20 rounds.
[2]	valid_0's mape: 2.4479
[3]	valid_0's mape: 2.4224
[4]	valid_0's mape: 2.39706
[5]	valid_0's mape: 2.37202
[6]	valid_0's mape: 2.34737
[7]	valid_0's mape: 2.32316
[8]	valid_0's mape: 2.33192
[9]	valid_0's mape: 2.30762
[10]	valid_0's mape: 2.28369
[11]	valid_0's mape: 2.2601
[12]	valid_0's mape: 2.26852
[13]	valid_0's mape: 2.24508
[14]	valid_0's mape: 2.2218
[15]	valid_0's mape: 2.19875
[16]	valid_0's mape: 2.17623
[17]	valid_0's mape: 2.15428
[18]	valid_0's mape: 2.13219
[19]	valid_0's mape: 2.11028
[20]	valid_0's mape: 2.08854
[21]	valid_0's mape: 2.09413
[22]	valid_0's mape: 2.07252
[23]	valid_0's mape: 2.05114
[24]	valid_0's mape: 2.0303
[25]	valid_0's mape: 2.0096
[26]	valid_0's mape: 1.98923
[27]	valid_0's mape: 1.96903
[28]	valid_0's mape: 1.98013
[29]	valid_0's mape: 1.9601
[30]	valid_0's mape: 1.94009
[31]	valid_0's mape: 1.94146
[32]	valid_0's mape: 1.92198
[33]	valid_0's mape: 1.90247
[34]	

[271]	valid_0's mape: 0.335048
[272]	valid_0's mape: 0.335277
[273]	valid_0's mape: 0.335574
Early stopping, best iteration is:
[253]	valid_0's mape: 0.332163


In [15]:
param_gbdt = {'num_leaves':75, 'num_trees':1200, 'objective':'regression_l1','learning_rate':0.05,'num_threads':-1,
        'feature_fraction':0.75,'bagging_fraction':0.75,'bagging_freq':1,'max_bin':20,'gaussian_eta':1,
        'lambda_l1':10,'boosting':'gbdt','device':'cpu'}
gbdt = lgb.train(param_gbdt, dtrain_meta, feval = mape_eval,valid_sets=[dval_meta],verbose_eval=True,num_boost_round=2500,early_stopping_rounds=20)

[1]	valid_0's mape: 2.23905
Train until valid scores didn't improve in 20 rounds.
[2]	valid_0's mape: 2.00923
[3]	valid_0's mape: 1.80683
[4]	valid_0's mape: 1.62806
[5]	valid_0's mape: 1.4682
[6]	valid_0's mape: 1.32632
[7]	valid_0's mape: 1.2039
[8]	valid_0's mape: 1.0935
[9]	valid_0's mape: 0.995217
[10]	valid_0's mape: 0.911564
[11]	valid_0's mape: 0.837056
[12]	valid_0's mape: 0.769925
[13]	valid_0's mape: 0.712393
[14]	valid_0's mape: 0.661751
[15]	valid_0's mape: 0.618142
[16]	valid_0's mape: 0.580644
[17]	valid_0's mape: 0.549535
[18]	valid_0's mape: 0.519968
[19]	valid_0's mape: 0.495378
[20]	valid_0's mape: 0.474066
[21]	valid_0's mape: 0.455738
[22]	valid_0's mape: 0.441001
[23]	valid_0's mape: 0.429042
[24]	valid_0's mape: 0.419205
[25]	valid_0's mape: 0.411675
[26]	valid_0's mape: 0.405314
[27]	valid_0's mape: 0.400771
[28]	valid_0's mape: 0.397168
[29]	valid_0's mape: 0.393932
[30]	valid_0's mape: 0.391903
[31]	valid_0's mape: 0.391048
[32]	valid_0's mape: 0.389959
[33]	v

In [27]:
param = {'max_depth':8, 'eta':0.01, 'silent':0, 'objective':'reg:linear', 'nthread':8,'lambda':3,
        'subsample ':0.8}
num_round = 80
eval_list = [(dval_meta_xgb,'eval')]
bst = xgb.train(param, dtrain_meta_xgb,num_round,evals=eval_list,feval=mape_eval_xgb,verbose_eval=20)

[0]	eval-mape:0.898877
[20]	eval-mape:0.684715
[40]	eval-mape:0.538164
[60]	eval-mape:0.448445


In [26]:
param_dart = {'num_leaves':50, 'num_trees':1200,'learning_rate':0.005,'objective':'regression_l1',
              'num_threads':8,'feature_fraction':0.65,'bagging_fraction':0.6,'gaussian_eta':1,
              'bagging_freq':1,'max_bin':20,'lambda_l1':10,'boosting':'dart','device':'cpu',
             'min_data_in_leaf':300}
stk_dart = lgb.train(param_dart, dtrain_meta,feval = mape_eval,valid_sets=[dval_meta],verbose_eval=True,num_boost_round=1500,early_stopping_rounds=20)

array([ 6.55224228,  6.55224228,  6.55224228, ...,  1.70903707,
        1.70903707,  1.70903707], dtype=float32)

In [ ]:
dtrain_stk = lgb.Dataset(stk_train_data,stk_train_y,feature_name=col_list,categorical_feature=['link_ID','class'])
dval_stk = lgb.Dataset(stk_val_data,stk_val_y,feature_name=col_list,categorical_feature=['link_ID','class'])
dtrain_stk_xgb = xgb.DMatrix(stk_train_data,stk_train_y,feature_names=col_list)
dval_stk_xgb = xgb.DMatrix(stk_val_data,stk_val_y,feature_names=col_list)